In [1]:
# Environment Preparation
import os
import sys
import shutil

import pandas as pd
import numpy as np
import arcpy
import pytz

# Setting working directory (CWD) and reading the dataset
cwd = os.getcwd()

file = cwd + '\\' + 'annarbor_gps.dat'

# processing workspace ------------------------------
strLocalDB =  cwd + "\\" "temp_db.gdb" ## need for local processing only
if os.path.isdir(strLocalDB):   # overwriting previous geodatabase
            shutil.rmtree(strLocalDB)
            print ("removed! past processing geodatabase")
else:
    pass

## testing local FGDB and assigning to the "localDB" variable
localDB = arcpy.CreateFileGDB_management(cwd, "temp_db")

## localDB = "in_memory"  # use for in memory workspace PROD!
localDBloc = str(localDB)
print (localDBloc)
arcpy.env.workspace = strLocalDB
# ------------------------------ local processing workspace building end 


print(file)
# data processig pandas ------------------------------
try:

    ## create dataframe out of DAT file, use the position of the columns for import
    df = pd.read_table(file, delimiter='\t', header=None, low_memory=False, index_col=False, names=[0,1,2,3,4,5,6,7,8,9,10,11,12])
    print(df)

    ## add a new field to a dataframe
    df["13"] = None
    df["14"] = None
    
    ## rename the fields (using position)
    df.columns =["TS", "Badge", "Handheld", "LAT", "LON", "ALT", "EVENT","D","A","S","P","B", "TD", "Timestamp","tsStrD"]

    ## remove all unneeded fields
    del df["ALT"]
    del df["D"]
    del df["A"]
    del df["S"]
    del df["P"]
    del df["B"]

    ## add new string date field, set to 0
    df['tsStrD'] = '0'
    print(df)

    ## replace null unix times with 0
    df["TD"] = df["TD"].replace(np.nan, 0)
    df["TS"] = df["TS"].replace(np.nan, 0)
    print(df)

    ## add time delay: create actual epoch time by adding up TS and TD
    sum_col = df["TS"] + df["TD"] # adding rover GPS time and GPS coordinates sync with system
    df["Timestamp"] = sum_col
    print("...added delay")
    
    ## convert dates: convert epoch time to dates
    df['Timestamp']=(pd.to_datetime(df['Timestamp'],unit='s'))
    print(df)
    print("...convert dates")
    
    ## timezone management: converse epoch time into Eastern Standard Time (a time zone qualifier needs to be managed so it is returned to the client as EST)
    df['Timestamp']=(pd.to_datetime(df['Timestamp'],unit='s')).dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    df['tsStrD'] = df['Timestamp'].astype(str) # convert data type into string
    df['tsStrD'] = df['tsStrD'].str[:-6] # remove the last six digits
    print('  timezone management')
    print(df)
    df['Timestamp'] = df['tsStrD']
    print("...convert timezone complete")

    ## remove unneeded fields from the dataframe
    del df["TS"]
    del df["TD"]
    del df["tsStrD"]
    
    ## set index for rows in the dataframe
    df.index.names = ['id'] 
    print(df)

    ## convert to a .csv file
    csvOut = cwd + "\\" + "csvAnnArborLocations.csv"  
    print(csvOut)
    print("...converting to csv")
    
    ## export dataframe to a .csv table
    csvTbl = df.to_csv(csvOut)
    print("...pandas data convert complete")
    pass

# find error that did not allowed exporting dataframe to CSV
except Exception as e:
    tb = sys.exc_info()[2]
    lineerror = "failed at \n" "Line %i" % tb.tb_lineno
    print (str(e))
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print (lineerror)
    print (str(tb))
     
    print("pandas data issue")

    sys.exit()
# ------------------------------ data processig pandas end     


# GIS Precessing Begins ------------------------------   
csvRaw = localDBloc + '\\' + "parkingCitationsRaw"

try:

    csvImport = arcpy.management.CopyRows(csvOut, csvRaw, '')
    ## create a filter: filter out records with "event type" coded as M or T
    csvFilter = arcpy.conversion.TableToTable(csvImport, localDBloc, "parkingCitationsSel", "EVENT IN ('M', 'T')",)
    print("...imported table")
    pass

    ## geocode
    ### Process: XY Table To Point (XY Table To Point) (management)
    geocode = arcpy.management.XYTableToPoint(in_table="parkingCitationsSel", out_feature_class="parkingCitationsSel_XYTableToPoint", x_field="LON", y_field="LAT")
    print("...geocode")
    pass

    ## further filter
    ### further select August
    parkingCitationsSel_XYTableToPoint_08 = arcpy.SelectLayerByAttribute_management(in_layer_or_view="parkingCitationsSel_XYTableToPoint", selection_type="NEW_SELECTION", where_clause="Timestamp >= timestamp '2021-08-19 10:29:22' And Timestamp <= timestamp '2021-08-31 17:36:17'", invert_where_clause="")
    ### Write the selected features to a new feature class
    arcpy.CopyFeatures_management(parkingCitationsSel_XYTableToPoint_08, "parkingCitationsSel_XYTableToPoint_08")
    
    ### further select September
    parkingCitationsSel_XYTableToPoint_09 = arcpy.SelectLayerByAttribute_management(in_layer_or_view="parkingCitationsSel_XYTableToPoint", selection_type="NEW_SELECTION", where_clause="Timestamp >= timestamp '2021-09-01 09:28:51' And Timestamp <= timestamp '2021-09-30 17:22:29'", invert_where_clause="")
    ### Write the selected features to a new feature class
    arcpy.CopyFeatures_management(parkingCitationsSel_XYTableToPoint_09, "parkingCitationsSel_XYTableToPoint_09")
    
    ### further select October
    parkingCitationsSel_XYTableToPoint_10 = arcpy.SelectLayerByAttribute_management(in_layer_or_view="parkingCitationsSel_XYTableToPoint", selection_type="NEW_SELECTION", where_clause="Timestamp >= timestamp '2021-10-01 09:20:55' And Timestamp <= timestamp '2021-10-30 16:24:40'", invert_where_clause="")
    ### Write the selected features to a new feature class
    arcpy.CopyFeatures_management(parkingCitationsSel_XYTableToPoint_10, "parkingCitationsSel_XYTableToPoint_10")
    
    ### further select November
    parkingCitationsSel_XYTableToPoint_11 = arcpy.SelectLayerByAttribute_management(in_layer_or_view="parkingCitationsSel_XYTableToPoint", selection_type="NEW_SELECTION", where_clause="Timestamp >= timestamp '2021-11-01 10:21:06' And Timestamp <= timestamp '2021-11-30 17:15:09'", invert_where_clause="")
    ### Write the selected features to a new feature class
    arcpy.CopyFeatures_management(parkingCitationsSel_XYTableToPoint_11, "parkingCitationsSel_XYTableToPoint_11")
    print("...further filter")
    pass

# find and print lines causing error
except Exception as e:
    tb = sys.exc_info()[2]
    lineerror = "failed at \n" "Line %i" % tb.tb_lineno
    print (str(e))
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print (lineerror)
    print (str(tb))
     
    print("GIS data Issue")
    
    sys.exit()
# ------------------------------ GIS Precessing Ends

# print the csvOut dataframe    
print(csvOut)

# remove CSV processing pandas staging file
# os.remove(csvOut)

arcpy.management.Project("parkingCitationsSel_XYTableToPoint_08", r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\parkingCitationsSel_XYTableToPoint_08_Project", 'PROJCS["WGS_1984_UTM_Zone_16N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("parkingCitationsSel_XYTableToPoint_09", r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\parkingCitationsSel_XYTableToPoint_09_Project", 'PROJCS["WGS_1984_UTM_Zone_16N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("parkingCitationsSel_XYTableToPoint_10", r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\parkingCitationsSel_XYTableToPoint_10_Project", 'PROJCS["WGS_1984_UTM_Zone_16N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("parkingCitationsSel_XYTableToPoint_11", r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\parkingCitationsSel_XYTableToPoint_11_Project", 'PROJCS["WGS_1984_UTM_Zone_16N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

KernelDensity_08 = arcpy.sa.KernelDensity("parkingCitationsSel_XYTableToPoint_08_Project", "NONE", 1, None, "SQUARE_KILOMETERS", "DENSITIES", "PLANAR", None); KernelDensity_08.save(r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\KernelDensity_08")
KernelDensity_09 = arcpy.sa.KernelDensity("parkingCitationsSel_XYTableToPoint_09_Project", "NONE", 1, None, "SQUARE_KILOMETERS", "DENSITIES", "PLANAR", None); KernelDensity_09.save(r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\KernelDensity_09")
KernelDensity_10 = arcpy.sa.KernelDensity("parkingCitationsSel_XYTableToPoint_10_Project", "NONE", 1, None, "SQUARE_KILOMETERS", "DENSITIES", "PLANAR", None); KernelDensity_10.save(r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\KernelDensity_10")
KernelDensity_11 = arcpy.sa.KernelDensity("parkingCitationsSel_XYTableToPoint_11_Project", "NONE", 1, None, "SQUARE_KILOMETERS", "DENSITIES", "PLANAR", None); KernelDensity_11.save(r"D:\ArcGIS\Projects\MyProject_ParkingTickets\MyProject_ParkingTickets.gdb\KernelDensity_11")

D:\ArcGIS\Projects\MyProject_ParkingTickets\temp_db.gdb
D:\ArcGIS\Projects\MyProject_ParkingTickets\annarbor_gps.dat
                 0      1   2         3   ...   9    10    11     12
0        1629323960  826.0  13  42.28202  ...  0.0    n   0.0    NaN
1        1629383197  826.0  13  42.28381  ...  0.0  NaN   NaN    NaN
2        1629383202  826.0  13  42.28381  ...  0.0  NaN   NaN    NaN
3        1629383217  826.0  13  42.28383  ...  0.0  NaN   NaN    NaN
4        1629383232  826.0  13  42.28386  ...  0.8  NaN  90.0    NaN
...             ...    ...  ..       ...  ...  ...  ...   ...    ...
1608137  1638314666  826.0  13  42.28803  ...  0.0    n   0.0  116.0
1608138  1638314797  826.0  13  42.28204  ...  0.0    n   0.0    NaN
1608139  1638304392  816.0   7  42.27951  ...  0.0  NaN   NaN    NaN
1608140  1638304397  816.0   7  42.27951  ...  0.0  NaN   NaN    NaN
1608141  1638304407  816.0   7  42.27951  ...  0.0  NaN   NaN    NaN

[1608142 rows x 13 columns]
                 TS  Badge